In [1]:
# !pip install ffmpeg-python

Collecting ffmpeg-python

  Downloading ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  
Requirement already satisfied: future in c:\programdata\anaconda3\lib\site-packages (from ffmpeg-python) (0.18.2)

Installing collected packages: ffmpeg-python

Successfully installed ffmpeg-python-0.2.0

In [41]:
import ffmpeg
import os
import subprocess
import re
import pandas as pd
import numpy as np

In [22]:
source = "TestFile.mp3"
output = "OutFile.mp3"
check = "TestFile2.mp3"

In [4]:
spd = 1.5 # output speed
vol = 1.0 # volume rise ratio if any
bit = 16 # BitRate: 16K
frq = 24000 # sampling stream frequency

#Temporary parameters
src, out = source, output
# commands
stg1 = f'ffmpeg -i {src} -filter:a "atempo={spd},volume={vol}" -q:a 100 {out}'
stg2 = f"ffmpeg -i {src} -ac 1 -b:a {bit}k -ar {frq} -write_xing 0 {output}"

### Finding silence and intervals, from this post
https://stackoverflow.com/questions/14590279/error-nameerror-name-subprocess-is-not-defined

In [12]:
command = f"ffmpeg -i {src} -af silencedetect=n=-30dB:d=0.25 -f null - "
out = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

stdout, stderr = out.communicate()
stdout

b"ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers\r\n  built with gcc 10.2.1 (GCC) 20200726\r\n  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cuda-llvm --enable-cuvid --enable-

In [21]:
splits = str(stdout).split("100 pcm_s16le\\r\\n")[1].split("\\r\\nsize=")[0]
splits = splits.split("\\r\\n")
splits

['[silencedetect @ 0000021c52e867c0] silence_start: 14.1607',
 '[silencedetect @ 0000021c52e867c0] silence_end: 14.5508 | silence_duration: 0.390113',
 '[silencedetect @ 0000021c52e867c0] silence_start: 14.5509',
 '[silencedetect @ 0000021c52e867c0] silence_end: 14.8773 | silence_duration: 0.32644',
 '[silencedetect @ 0000021c52e867c0] silence_start: 53.0442',
 '[silencedetect @ 0000021c52e867c0] silence_end: 53.3589 | silence_duration: 0.314671',
 '[silencedetect @ 0000021c52e867c0] silence_start: 67.8339',
 '[silencedetect @ 0000021c52e867c0] silence_end: 68.1681 | silence_duration: 0.334218',
 '[silencedetect @ 0000021c52e867c0] silence_start: 114.954',
 '[silencedetect @ 0000021c52e867c0] silence_end: 115.233 | silence_duration: 0.279206',
 '[silencedetect @ 0000021c52e867c0] silence_start: 131.051',
 '[silencedetect @ 0000021c52e867c0] silence_end: 131.385 | silence_duration: 0.333991',
 '[silencedetect @ 0000021c52e867c0] silence_start: 143.995',
 '[silencedetect @ 0000021c52e867

#### Let's apply the same to other file, to see the output format

In [23]:
def get_silence_times(src):
    command = f"ffmpeg -i {src} -af silencedetect=n=-30dB:d=0.25 -f null - "
    out = subprocess.Popen(command,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.STDOUT)
    stdout, stderr = out.communicate()
    splits = str(stdout).split("100 pcm_s16le\\r\\n")[1].split(
        "\\r\\nsize=")[0]
    splits = splits.split("\\r\\n")
    return splits

In [24]:
get_silence_times(check)

['[silencedetect @ 000001e510f2b080] silence_start: 0',
 '[silencedetect @ 000001e510f2b080] silence_end: 0.766417 | silence_duration: 0.766417',
 '[silencedetect @ 000001e510f2b080] silence_start: 3.41925',
 '[silencedetect @ 000001e510f2b080] silence_end: 5.25726 | silence_duration: 1.838',
 '[silencedetect @ 000001e510f2b080] silence_start: 6.18304',
 '[silencedetect @ 000001e510f2b080] silence_end: 6.96218 | silence_duration: 0.779138',
 '[silencedetect @ 000001e510f2b080] silence_start: 9.79152',
 '[silencedetect @ 000001e510f2b080] silence_end: 10.9414 | silence_duration: 1.14986',
 '[silencedetect @ 000001e510f2b080] silence_start: 11.2242',
 '[silencedetect @ 000001e510f2b080] silence_end: 12.2511 | silence_duration: 1.02696',
 '[silencedetect @ 000001e510f2b080] silence_start: 17.3094',
 '[silencedetect @ 000001e510f2b080] silence_end: 18.0719 | silence_duration: 0.762426',
 '[silencedetect @ 000001e510f2b080] silence_start: 18.7098',
 '[silencedetect @ 000001e510f2b080] silen

It appears that there are 3 lines at the end of outputs that are not related to `silencedetect` 

## Trying regex

In [35]:
str_out = str(stdout) 
# reg = r'(silence_start: (\d+.\d+)).*(silence_end: (\d+.\d+)).*(silence_duration: (\d+.\d+))'
# reg = r'(silence_start: (\d+.\d+))'
# reg = r'silence_start: (\d+.\d+)'
s_start = re.findall(r'silence_start: (\d+.\d+)', str_out)
s_end = re.findall(r'silence_end: (\d+.\d+)', str_out)
s_duration = re.findall(r'silence_duration: (\d+.\d+)', str_out)
s_start = map(float, s_start)
s_end= map(float, s_end)
s_duration= map(float, s_duration)
results = list(zip(s_start, s_end, s_duration))
results

[(14.1607, 14.5508, 0.390113),
 (14.5509, 14.8773, 0.32644),
 (53.0442, 53.3589, 0.314671),
 (67.8339, 68.1681, 0.334218),
 (114.954, 115.233, 0.279206),
 (131.051, 131.385, 0.333991),
 (143.995, 144.25, 0.255125),
 (168.528, 168.838, 0.30966),
 (174.488, 174.809, 0.321088),
 (186.484, 186.759, 0.274921),
 (246.877, 247.2, 0.322494),
 (354.82, 355.123, 0.302766),
 (381.662, 382.15, 0.487982),
 (387.647, 387.954, 0.307256),
 (391.426, 391.968, 0.541338),
 (411.457, 411.745, 0.287506),
 (413.169, 413.475, 0.305624),
 (414.698, 415.037, 0.338889),
 (415.894, 416.155, 0.260975),
 (417.332, 417.977, 0.644603),
 (421.379, 421.786, 0.406893),
 (424.438, 424.878, 0.439841),
 (439.395, 439.699, 0.304059)]

## Splitting
The file `TestFile.mp3` duration is 7:21. (441 seconds)

Consider splitting the file every 60 seconds, 

i.e. @60, 120, 180, 240, 300, 360, 420 seconds

If the remaining portion is less than 30 seconds, then it should be merged to the last portion.

***We need to import pandas to deal with the table faster***

In [39]:
results_df = pd.DataFrame(
    results, columns=['silence_start', 'silence_end', 'silence_duration'])
results_df.head(3)

,silence_start,silence_end,silence_duration
0,14.1607,14.5508,0.390113
1,14.5509,14.8773,0.326440
2,53.0442,53.3589,0.314671


We need to split at the middle of the silence period,
 i.e., @ `silence_start + silence_duration / 2`

In [40]:
results_df[
    'silence_mid'] = results_df.silence_start + results_df.silence_duration / 2

results_df.head(3)

,silence_start,silence_end,silence_duration,silence_mid
0,14.1607,14.5508,0.390113,14.355757
1,14.5509,14.8773,0.326440,14.714120
2,53.0442,53.3589,0.314671,53.201535


looping through the `silence_mid` to find the neasest value to each of our splitting points

***we need numpy to create a list of stopping points***

In [61]:
# From this post:
#     https://stackoverflow.com/a/64582429/5820024
end_time = float(ffmpeg.probe(src)['format']['duration'])
end_time

440.502857

In [69]:
split_interval = 60
split_arr = np.arange(split_interval, int(end_time), split_interval)
split_arr

array([ 60, 120, 180, 240, 300, 360, 420])

getting an array of `silence_mid`

In [45]:
silence_mid = results_df.silence_mid.to_numpy()
silence_mid 

array([ 14.3557565,  14.71412  ,  53.2015355,  68.001009 , 115.093603 ,
       131.2179955, 144.1225625, 168.68283  , 174.648544 , 186.6214605,
       247.038247 , 354.971383 , 381.905991 , 387.800628 , 391.696669 ,
       411.600753 , 413.321812 , 414.8674445, 416.0244875, 417.6543015,
       421.5824465, 424.6579205, 439.5470295])

In [53]:
# for i, split_time in enumerate(split_arr):
i=0
success_array =[]
for j, silence_time in enumerate(silence_mid):
    if silence_time < split_arr[i]:
        continue
    else:
        # compare which item is closer to our suggested split time
        prev_silence = silence_mid[j-1]
        if silence_time-split_arr[i] > split_arr[i] - prev_silence:
            if prev_silence not in success_array:
                success_array.append(prev_silence)
            else:
                success_array.append(silence_time)
        else:
            success_array.append(silence_time)
        # move to the next time in the split_arr
        if i< len(split_arr)-1:
            i +=1
        else:
            break
success_array

[53.2015355,
 115.09360299999999,
 174.648544,
 247.038247,
 354.971383,
 381.905991,
 421.5824465]

Getting the difference between each two intervals, to get the lengths of split files
1. add zero before the array
1. add the lenght of the mp3 file after the array


In [75]:
dur_array= [0] + success_array+[end_time]
# dur_array= success_array+[end_time]
dur_array

[0,
 53.2015355,
 115.09360299999999,
 174.648544,
 247.038247,
 354.971383,
 381.905991,
 421.5824465,
 440.502857]

In [76]:
difference_array = np.diff(np.array(dur_array))
difference_array

array([ 53.2015355,  61.8920675,  59.554941 ,  72.389703 , 107.933136 ,
        26.934608 ,  39.6764555,  18.9204105])

### Merging small parts
Small part is considdered small if it is less than `x` seconds

The small part should be merged to the next part, so that the sum does not exceeds `y` seconds

In [97]:
min_duration = 0.8 * split_interval
max_duration = 2.5 * split_interval
min_duration, max_duration

(48.0, 150.0)

In [110]:
# Check each element for the forward round
def merge_small_durations(diff_arr, min_dur, max_dur):
    dur_array_forward = []
    flag_raised=False # the flag will be raised if we merges occur
    for i, dur in enumerate(diff_arr):
        if flag_raised: 
            flag_raised=False
            continue
        if (dur < min_dur) & (dur > 0):
            if i + 1 < len(diff_arr):
                next_diff = diff_arr[i + 1]
#                 if next_diff==0:
#                     next_diff = diff_arr[i + 2]
                if dur + next_diff < max_dur:
                    # dur_array_forward.append(0)
                    dur_array_forward.append(dur + next_diff)
                    flag_raised=True
        else:
            dur_array_forward.append(dur)
    return dur_array_forward


fwd_arr = merge_small_durations(difference_array, min_duration, max_duration)
fwd_arr

[53.2015355,
 61.89206749999999,
 59.554941,
 72.38970300000003,
 107.93313599999999,
 66.6110635]

In [111]:
bwd_arr = merge_small_durations(fwd_arr[::-1], min_duration,
                                max_duration)[::-1]
bwd_arr

[53.2015355,
 61.89206749999999,
 59.554941,
 72.38970300000003,
 107.93313599999999,
 66.6110635]

In [112]:
# getting the modified success_array
print(success_array)
print(difference_array)
print(bwd_arr)

list(zip(success_array, bwd_arr))


[53.2015355, 115.09360299999999, 174.648544, 247.038247, 354.971383, 381.905991, 421.5824465]
[ 53.2015355  61.8920675  59.554941   72.389703  107.933136   26.934608
  39.6764555  18.9204105]
[53.2015355, 61.89206749999999, 59.554941, 72.38970300000003, 107.93313599999999, 66.6110635]


[(53.2015355, 53.2015355),
 (115.09360299999999, 61.89206749999999),
 (174.648544, 59.554941),
 (247.038247, 72.38970300000003),
 (354.971383, 107.93313599999999),
 (381.905991, 66.6110635)]

In [113]:
def merge_small_durations(diff_arr, min_dur, max_dur, fwd=True):
    dur_arr_mod = []
    flag_raised = False  # the flag will be raised if we merges occur
    for i, dur in enumerate(diff_arr):
        if flag_raised:
            flag_raised = False
            continue
        if (dur < min_dur) & (dur > 0):
            # if dur < min_dur:
            if i + 1 < len(diff_arr):
                next_diff = diff_arr[i + 1]
                if dur + next_diff < max_dur:
                    if fwd: dur_arr_mod.append(0)
                    dur_arr_mod.append(dur + next_diff)
                    if not fwd: dur_arr_mod.append(0)
                    flag_raised = True
            else:
                dur_arr_mod.append(dur)
        else:
            dur_arr_mod.append(dur)
    return dur_arr_mod

In [117]:
fwd_arr = merge_small_durations(difference_array, min_duration, max_duration)

bwd_arr = merge_small_durations(fwd_arr[::-1],
                                min_duration,
                                max_duration,
                                fwd=False)[::-1]

# getting the modified success_array
for ary in [success_array, dur_array, list(difference_array), fwd_arr, bwd_arr]:
    print(len(ary), ary)


7 [53.2015355, 115.09360299999999, 174.648544, 247.038247, 354.971383, 381.905991, 421.5824465]
9 [0, 53.2015355, 115.09360299999999, 174.648544, 247.038247, 354.971383, 381.905991, 421.5824465, 440.502857]
8 [53.2015355, 61.89206749999999, 59.554941, 72.38970300000003, 107.93313599999999, 26.93460799999997, 39.67645550000003, 18.920410500000003]
8 [53.2015355, 61.89206749999999, 59.554941, 72.38970300000003, 107.93313599999999, 0, 66.6110635, 18.920410500000003]
8 [53.2015355, 61.89206749999999, 59.554941, 72.38970300000003, 107.93313599999999, 0, 0, 85.531474]


`[53.2015355, 61.89206749, 59.554941, 72.389703, 107.9331359, 26.93460797, 39.67645550, 18.920410500]

[53.2015355, 61.89206749999999, 59.554941, 72.38970300000003, 107.93313599999999, 0, 0, 85.531474]

[53.2015355, 115.09360299999999, 174.648544, 247.038247, 354.971383, 381.905991, 421.5824465, 440.502857]

[0, 53.2015355, 115.09360299999999, 174.648544, 247.038247, 354.971383, 381.905991, 421.5824465, 440.502857]`


In [120]:
list(zip(dur_array[1:], list(difference_array), bwd_arr))

[(53.2015355, 53.2015355, 53.2015355),
 (115.09360299999999, 61.89206749999999, 61.89206749999999),
 (174.648544, 59.554941, 59.554941),
 (247.038247, 72.38970300000003, 72.38970300000003),
 (354.971383, 107.93313599999999, 107.93313599999999),
 (381.905991, 26.93460799999997, 0),
 (421.5824465, 39.67645550000003, 0),
 (440.502857, 18.920410500000003, 85.531474)]

In [121]:
final_stops = [x for i, x in enumerate(dur_array[1:]) if bwd_arr[i]!=0]
final_stops

[53.2015355,
 115.09360299999999,
 174.648544,
 247.038247,
 354.971383,
 440.502857]

In [122]:
# remove the last element as it is the file length

final_stops = final_stops[:-1]
final_stops

[53.2015355, 115.09360299999999, 174.648544, 247.038247, 354.971383]

In [129]:
final_durations = list(
    np.round(np.diff(np.array([0] + final_stops + [end_time])), 3))
final_durations

[53.202, 61.892, 59.555, 72.39, 107.933, 85.531]

In [ ]:
# adding a function to round values of a list
def round_list_values(the_list, rounding=3):
    return list(np.round(the_list, rounding))